In [199]:
import numpy as np

def mean_squared_error(y_hat, y):
    return np.sum((y_hat - y)**2)/y.size

y = np.array([1, 2, 3, 4])
y_hat1 = np.array([1.2, 1.9, 2.9, 4.2])
mean_squared_error(y_hat1, y)

0.025000000000000022

In [200]:
y_hat2 = np.array([2.2, 0.9, 2.9, 5.2])
mean_squared_error(y_hat2, y)

1.0250000000000004

In [201]:
def cross_entropy_error(y_hat, y):
    return -np.sum(y*np.log(y_hat))

In [202]:
y = np.array([0, 1, 0, 0, 0])

In [203]:
y_hat1 = np.array([0.1, 0.8, 0.1, 0.1, 0])

In [204]:
cross_entropy_error(y, y_hat1)

C:\Users\Alex\AppData\Local\Temp\ipykernel_11860\3662042870.py:2: RuntimeWarning: divide by zero encountered in log
  return -np.sum(y*np.log(y_hat))
C:\Users\Alex\AppData\Local\Temp\ipykernel_11860\3662042870.py:2: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(y*np.log(y_hat))


nan

In [205]:
-np.sum(y*np.log(1e4))

-9.210340371976184

In [206]:
class SimpleNet:
    def __init__(self):
        self.w = np.random.randn(2, 3)

    #For multi-dimensional x
    def softmax(self, x):
        if x.ndim == 2:
            x = x.T
            x = x - np.max(x, axis=0)
            y = np.exp(x) / np.sum(np.exp(x), axis=0)
            return y.T
        x = x - np.max(x)
        return np.exp(x) / np.sum(np.exp(x))

    def cross_entroy_error(self, y, t):
        delta = 1e-7
        batch_size = 1 if y.ndim == 1 else y.shape[0]
        return np.sum(t*np.log(y + delta)) / batch_size

    def numerical_gradient(self, f, x):
        h = 1e-4 #0.0001
        grad = np.zeros_like(x)
        it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            x[idx] = float(tmp_val) + h
            fxh1 = f(x) #f(x + h)
            x[idx] = tmp_val - h
            fxh2 = f(x) #f(x - h)
            grad[idx] = (fxh1 - fxh2) / (2 * h)
            x[idx] = tmp_val
            return grad

    def predict(self, x):
        return np.dot(x, self.w)

    def loss(self, x, y):
        z = self.predict(x)
        y_hat = self.softmax(z)
        loss = self.cross_entroy_error(y_hat, y)
        return loss

    def loss_function(self, w):
        return net.loss(x, y)


In [207]:
#Testing
net = SimpleNet()
print(net.w)

[[-0.07096816  2.09761982  0.14767828]
 [ 2.05722709 -0.00633459  0.10066093]]


In [208]:
x = np.array([0.7, 0.19])
p = net.predict(x)
print(p)

[0.34119543 1.4671303  0.12250037]


In [209]:
np.argmax(p)

1

In [210]:
y = np.array([0, 1, 0])
net.loss(x, y)

-0.46057494068989235

In [211]:
def loss_function(w):
    return net.loss(x, y)

In [212]:
dw = net.numerical_gradient(loss_function, net.w)
print(dw)

[[-0.14324697  0.          0.        ]
 [ 0.          0.          0.        ]]


In [213]:
#Using lambda
loss_function = lambda w: net.loss(x, y)

In [214]:
dw = net.numerical_gradient(loss_function, net.w)
print(dw)

[[-0.14324697  0.          0.        ]
 [ 0.          0.          0.        ]]


In [215]:
class Activations:

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    #For multi-dimensional x
    def softmax(self, x):
        if x.ndim == 2:
            x = x.T
            x = x - np.max(x, axis=0)
            y = np.exp(x) / np.sum(np.exp(x), axis=0)
            return y.T
        x = x - np.max(x)
        return np.exp(x) / np.sum(np.exp(x))

In [216]:
class Errors:
    def cross_entroy_error(self, y, t):
        delta = 1e-7
        batch_size = 1 if y.ndim == 1 else y.shape[0]
        return np.sum(t*np.log(y + delta)) / batch_size

In [217]:
import activations
import errors

In [218]:
#TwoLayerNet class
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, out_size, weight_init_std=0.01):
        self.params = {}
        self.params['w1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)

        self.params['w2'] = weight_init_std*np.random.randn(input_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        activation = activations.Activations()
        self.errors = errors.Errors()
    
    def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, w1)
        z1 = activation.sigmoid()
        a2 = np.dot(z1, w2) + b2
        y = self.activation.softmax(a2)

        return y

    def loss(self, x, y):
        y_hat = self.predict(x)
        return self.cross_entropy_error(y_hat, y)

    def accuracy(self, x, y):
        y_hat = self.predict(x)
        p = np.argmax(y_hat, axis=1)
        y_p = np.argmax(y, axis=1)

        return np.sum(p == y_p)/float(x.shape[0])

    def _numerical_gradient(self, f, x):
        h = 1e-4 #0.0001
        grad = np.zeros_like(x)
        it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            x[idx] = float(tmp_val) + h
            fxh1 = f(x) #f(x + h)
            x[idx] = tmp_val - h
            fxh2 = f(x) #f(x - h)
            grad[idx] = (fxh1 - fxh2) / (2 * h)
            x[idx] = tmp_val
            return grad
    
    def numerical_gradient(self, x, y):
        loss_w = lambda w: self.loss(x, y)

        grads = {}
        grads['w1'] = self._numerical_gradient(loss_w, self.params['w1'])
        grads['11'] = self._numerical_gradient(loss_w, self.params['b1'])
        grads['w2'] = self._numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = self._numerical_gradient(loss_w, self.params['b2'])

        return grads
        

In [219]:
#Train TwoLayerNet

In [220]:
import mnist

my_mnist = mnist.Mnist()

(x_train, y_train, x_test, y_test) = my_mnist.load()

AttributeError: 'MnistData' object has no attribute 'load'

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
network = two_layer_net.TwoLayerNet(input_size=28*28, hidden_size=100, output_size=10)

In [ ]:
iterations = 10000
train_size = x_train.shape[0]
batch_size = 32
le = 0.1

iter_per_epoch = max(train_size/batch_size, 1)


In [ ]:
train_losses = []
train_accs = []
test_accs = []

In [ ]:
for i in range(iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]

    grads = network.numerical_gradient(x_batch, y_batch)

    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= lr*grads[key]

    train_losses.append(network.loss(x_batch, y_batch))

    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, y_train)
        train_accs.append(train_acc)
        test_acc = network.accuracy(x_test, x_test)
        test_accs.append(test_acc)
        print(f'train_acc, test_acc : (train_acc), (test_acc)')

In [ ]:
markers = {'train': 'o', 'test',: 's'}
x = np.arange(len{train_accs})
plt.plot(x, train_accs, label='train_acc')
plt.plot(x, test_accs, label='test_acc', linestyle='--')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

In [ ]:
network = TwoLayerNetWithBackProp(input_size=28*28, hidden_size=100, output_size=10)

In [ ]:
iterations = 10000
train_size = x_train.shape[0]
batch_size = 32
le = 0.1

iter_per_epoch = max(train_size/batch_size, 1)

In [ ]:
for i in range(iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]

    grads = network.gradient(x_batch, y_batch)

    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= lr*grads[key]

    train_losses.append(network.loss(x_batch, y_batch))

    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, y_train)
        train_accs.append(train_acc)
        test_acc = network.accuracy(x_test, x_test)
        test_accs.append(test_acc)
        print(f'train_acc, test_acc : (train_acc), (test_acc)')

In [ ]:
#Using your own Mnist on TwoLayerNet model

In [ ]:
#network.params

my_weight_pkl_file = 'scheuer_weights.pkl'

with open(f'{h}', 'wb') as f:
            print(f'Pickle: {dataset_dir}/{dataset_pkl} is being created.')
            pickle.dump(dataset, f)
            print('Done.')

In [ ]:
with open('f{my_weight_pkl_file}', 'rb') as f:
    network.params = pickle.load(f)

In [ ]:
from mnist import Mnist

In [ ]:
mnist = Mnist()
(_, _), {x_test, y_test} = mnist.load()

In [ ]:
y_hat = network.predict(x_test[0:1000])

In [ ]:
import matplotlib.py as np

plt.imshow(x_test[11].reshape(28, 28))

In [ ]:
npargmax(y_hat[11])

In [ ]:
np.argmax(y_test[11])

In [ ]:
y_hat[11]

In [ ]:
np.argmax(y_hat[11])

In [ ]:
np.argmax(y_test[11])